#EDA Process
EDA : Exploratory Data Analysis 탐색적 데이터 분석


------------------------------------------------------------------------------

1. 시각화를 통한 분포 확인
  - histogram, pie-chart, boxplot, qqplot(분포) 등

2. Summary Statistics : 데이터 분포 및 특이성 확인
  -  mean, sd, skewness, kurtosis, outliers, frequency 등

3. 독립변수 선정
  - 데이터 해석 상의 배경지식 반영
  - 상관성 높은 변수 제거 (비추)
  - 차원 축소 (설명력 높은 수준의 PCA)
  - 파생변수 생성

4. Data Filtering
  - Data type 확인(수치형 <> 범주형)
  - 중복 데이터 확인
  - 결측치 제거 및 중위값 대체
  - outlier 처리
  - 범주형 : one-hot encoding

5. 범주형 변수 처리

6. scaling
  - z-normalize, MinMax


------------------------------------------------------------------------------



1. 시각화를 통한 분포 확인
  - histogram, pie-chart, boxplot, qqplot(분포) 등

In [2]:
# 수치형 변수들의 분포 시각적 확인
f,ax=plt.subplots(4,3,figsize=(19,6),constrained_layout = True)

sns.distplot(df["A"],bins=20,ax=ax[0,0],color='orange');
sns.distplot(df["B"],bins=20,ax=ax[0,1],color='orange');
sns.distplot(df["C"],bins=20,ax=ax[0,2],color='orange');
sns.distplot(df["D"],bins=20,ax=ax[1,0],color='red');
sns.distplot(df["E"],bins=20,ax=ax[1,1],color='red');
sns.distplot(df["F"],bins=20,ax=ax[1,2],color='red');
sns.distplot(df["G"],bins=20,ax=ax[2,0],color='black');
sns.distplot(df["H"],bins=20,ax=ax[2,1],color='black');
sns.distplot(df["I"],bins=20,ax=ax[2,2],color='black');
sns.distplot(df["J"],bins=20,ax=ax[3,0],color='gray');
sns.distplot(df["K"],bins=20,ax=ax[3,1],color='gray');
sns.distplot(df["L"],bins=20,ax=ax[3,2],color='gray');

https://www.kaggle.com/code/galipsekeroglu/randomforest-explainableai-xai-eda-gridsearch

In [ ]:
# 독립변수 A에 따른 target 변수(price) 분포
plt.figure(figsize=(20,8))
g = sns.scatterplot(x='A',y='Price',data=df)

g.set_title('A VS Price Correlation',fontsize=20)
g.set_xlabel('Ar',fontsize=12)
g.set_ylabel('Price',fontsize=12)

xlabels = ['{:,.2f}'.format(x)+'k' for x in g.get_xticks()/10e3]
ylabels = ['{:,.2f}'.format(y)+'k' for y in g.get_yticks()/10e3]

g.set_xticklabels(xlabels)
g.set_yticklabels(ylabels)

2. Summary Statistics : 데이터 분포 및 특이성 확인
  -  mean, sd, skewness, kurtosis, outliers, frequency 등

In [4]:
df.head(n) # DataFrame 형태로 데이터 상위 n개 확인

df.info() # class, Column별 데이터 갯수 및 type 확인

df.describe().T

https://blog.naver.com/charzim0611/222922894275

3. 독립변수 선정
  - 데이터 해석 상의 배경지식 반영
  - 상관성 높은 변수 제거 (비추)
  - 차원 축소 (설명력 높은 수준의 PCA)
  - 파생변수 생성

In [ ]:
# 각 변수들간의 상관관계 파악, 시각화 -> heatmap
plt.subplots(figsize=(12,9))
sns.heatmap(df.corr(), annot=True, vmax=0.9, square=True,cmap='RdYlGn') # 변수간 상관관계 corr()
plt.show();

In [ ]:
# PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 0.9 이상의 강한 상관관계가 있는 변수들(A~Z)을 선형변환을 통해 차원 축소
col_PCA = ['A,B...,Z']
x = df[col_PCA]

scaler = StandardScaler()
scaler.fit(x)
scaled_X_train = scaler.transform(x)

pca = PCA(n_components=2) #PCA 객체 생성 (주성분 갯수 2개 생성)
pca.fit(scaled_X_train)
principalComponents = pca.transform(scaled_X_train)
principalDf = pd.DataFrame(data = principalComponents, columns = ['PCA_1', 'PCA_2'])
principalDf

pca.explained_variance_ratio_  # PCA1,PCA2 2개의 성분으로 cover하고 있는 설명력 확인

# 두 PCA1,PCA2 변수를 추가하고 사용한 변수는 제거한다.
df.insert(0,"PCA_2",principalDf.PCA_2,True) 
df.insert(0,"PCA_1",principalDf.PCA_1,True)
df = df.drop(col_PCA,axis=1)
df.info()

4. Data Filtering
  - Data type 확인(수치형 <> 범주형)
  - 중복 데이터 확인
  - 결측치 제거 및 중위값 대체
  - outlier 처리
  - 범주형 : one-hot encoding

년도, 시간 -> data type으로 합쳐

In [ ]:
# 1. null값들의 존재 유무 및 그 비율 확인
for column in df.columns:
  if df[column].isna().sum()!=0:
    missing = df[column].isna().sum()
    portion = (missing/df.shape[0])*100
    print(f"'{column}':number of missing value '{missing}' ==> '{portion:.3f}%'")
print('==========================================')

# 2. column마다의 null 갯수
data.isnull().sum()

# 3. null 유무 확인 (T/F)
df.isnull().values.any()

## null 비율이 큰 변수는 제거하고 작은 변수는 최빈값으로 대체
mode = df['A'].value_counts().index[0]
df['A'] = df['A'].fillna(mode)
df['B'] = df['B'].replace('-',mode)

## null인 값을 정교하게 예측해서 값을 대체할 수도 있음 <자료 참고>


In [ ]:
# 중복되는 데이터가 존재하는 확인
print(f"Data shape:{df.shape}")
df.drop_duplicates(inplace=True)
print(f"Data shape:{df.shape}")
print('==========================================')

# 중복 유무 확인
df.duplicated().any()

In [ ]:
# 1%의 이상치 제거
df = df.loc[np.where(df['A'] < df['A'].quantile(0.99))] # 변수A에 대해선 큰값 1% 제거
df.reset_index(drop=True, inplace= True)

df = df.loc[np.where(df['B'] > df['B'].quantile(0.01))]  # 변수B에 대해선 작은 값 1% 제거
df.reset_index(drop=True, inplace= True)

df.info()


5. 범주형 변수 : One-hot encoding

In [ ]:
# Categorical features 파악
for column in df.columns:
    if df[column].dtype == object:
        print(column)
        print(df[column].unique())
        print("")

# one-hot encoding
onehot = pd.get_dummies(df[CATEGORICAL], drop_first=False)

6. Scaling

In [ ]:
## 스케일링 함수
def z_normalize (train, list): # 대상이 될 dataset과 feature list를 받으면 normalization 해주는 함수. 이때 사용된 평균과 표준편차를 함께 return한다.
    data=train.loc[:, list]
    mean=data.mean()  # mean 저장
    std=data.std()    # standard deviation 저장
    data=(data-mean)/std  # normalization
    return data, [mean,std]

def minmax_scale (train, list):
    data=train.loc[:, list]
    min=data.min()    # min 저장
    max=data.max()    # max 저장
    data=(data-min)/(max-min)  # scale
    return data, [min,max]
